# 第3回 その2: 時系列データの可視化 (2)：直線近似
時系列データの近似直線を書いてみましょう。  


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/03"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/03/
# 現在のフォルダの中身を表示
%ls

`energy.csv` と`covid19.csv`というデータが表示されていることを確認してください。

## ステップ1: 直線近似
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`energy.csv` を読み込み，データをプロットします。  
このデータは，1970年から2000年までの，日本の一世帯あたりの電力消費量を記録したデータです。  
(出典：一世帯あたりの電力消費量の推移 https://www.ene100.jp/zumen/1-2-13 の一部を抜粋)

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('energy.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) として抽出する。
year = csv_data.loc[:, '年'].to_numpy()
data = csv_data.loc[:, '電力消費量(kWh)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(10,5))
plt.scatter(year, data, label='Real Data', color='b')
plt.xlabel('Year')
plt.ylabel('Power consumption [kWh]')
plt.legend()
plt.show()

このデータを線形関数 (y = ax + b)で近似します。  
近似関数の傾き(a)と切片(b)を計算する関数を以下に定義します。  

In [ ]:
def linear_approx(data):
  '''
      data: 線形直線で近似したいデータ
  '''
  N = np.size(data)
  y = np.array(data)
  x = np.arange(N) # x=[0, 1, ..., N-1]

  # 係数計算に必要な変数の計算
  xx = x * x
  xy = x * y
  
  # a と b の計算で共通する分母の計算
  bunbo = N * np.sum(xx) - np.sum(x)*np.sum(x)

  # a および b の計算
  a = (N*np.sum(xy) - np.sum(x)*np.sum(y)) / bunbo
  b = (np.sum(xx)*np.sum(y) - np.sum(xy)*np.sum(x)) / bunbo

  return (a, b)

上で定義した関数 `linear_approx` を使って，`data` の直線近似関数を計算します。

In [ ]:
# 直線近似関数のパラメータを求める
(a, b) = linear_approx(data)
print('approximate function: y = %fx + %f' % (a, b))

# 直線近似関数を作成
x = np.arange(np.size(data))
approx = a * x + b

近似結果によると，この期間おいては，毎月約 6kWhkずつ増加していることが分かります。   
実データと近似直線をプロットします。

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(year, data, label='Real Data', color='b')
plt.plot(year, approx, label='Approximate function', color='r')
plt.xlabel('Year')
plt.ylabel('Power consumption [kWh]')
plt.legend()
plt.show()

近似直線が実際の値に対してどの程度ずれているのか，計算してみましょう。

In [ ]:
# 平均二乗誤差(mean square error)
mse = np.mean((data - approx)*(data - approx))
# 平均平方根二乗誤差 (root mean square error) = mse の平方根
rmse = np.sqrt(mse)

print('MSE = %f' % (mse))
print('RMSE = %f' % (rmse))

平均で約 7.06 kWh 程度の誤差であることから，そこそこの精度で近似できているように見えます。